This workbook represent an end to end test of the Nillion network. It will test the following:
1. A user (patient) uploading their data in encrypted form and providing access to Monadic
2. Monadic giving compute access to Snipper, a third party
3. Snipper registering a program and running in on the patient's encrypted data

Before running this workbook, ensure Nillion Devnet is running. 

Install all Python prerequisites. 

In [ ]:
%pip install nada-dsl==0.6.0
%pip install py-nillion-client==0.6.0
%pip install nillion-python-helpers==0.2.3
%pip install python-dotenv==1.0.0


Import all necessary libraries

In [2]:
import py_nillion_client as nillion
from py_nillion_client import NodeKey, UserKey
from dotenv import load_dotenv
from nillion_python_helpers import get_quote_and_pay, create_nillion_client, create_payments_config
from cosmpy.aerial.client import LedgerClient
from cosmpy.aerial.wallet import LocalWallet
from cosmpy.crypto.keypairs import PrivateKey

Load the environment variables from a Devnet .env file

*Please make sure to replace the path with the correct path to your .env file*

In [ ]:
import os

home_dir = os.path.expanduser("~")
env_path = os.path.join(home_dir, ".config", "nillion", "nillion-devnet.env")

print(f"Loading environment variables from {env_path}")

load_dotenv(env_path)

for key, value in os.environ.items():
    if key.startswith("NILLION_"):
        print(f"{key}: {value}")

Read in some basic Nillion network information from the environment. 

In [ ]:
cluster_id = os.getenv('NILLION_CLUSTER_ID')
chain_id = os.getenv('NILLION_NILCHAIN_CHAIN_ID')
grpc_endpoint = os.getenv('NILLION_NILCHAIN_GRPC')

print(f"Cluster ID: {cluster_id}")
print(f"Chain ID: {chain_id}")
print(f"GRPC Endpoint: {grpc_endpoint}")

Each actor will need its own node key so we define a utility function here.

In [5]:
import uuid
def gen_node_key():
    return NodeKey.from_seed(uuid.uuid4().hex)

Setup for the Monadic actor

In [6]:
monadic_seed = "monadic_seed"
monadic_userkey = UserKey.from_seed(monadic_seed)
monadic_client = create_nillion_client(monadic_userkey, gen_node_key())
monadic_party_id = monadic_client.party_id
monadic_user_id = monadic_client.user_id

Setup for the Snipper actor

In [7]:
snipper_seed = "snipper_seed"
snipper_userkey = UserKey.from_seed(snipper_seed)
snipper_client = create_nillion_client(snipper_userkey, gen_node_key())
snipper_party_id = snipper_client.party_id
snipper_user_id = snipper_client.user_id

Setup for the Patient actor

In [8]:
patient_seed = "patient_seed"
patient_userkey = UserKey.from_seed(patient_seed)
patient_client = create_nillion_client(patient_userkey, gen_node_key())
patient_party_id = patient_client.party_id
patient_user_id = patient_client.user_id

Payments set up for all network actions

In [9]:
payments_config = create_payments_config(chain_id, grpc_endpoint)
payments_client = LedgerClient(payments_config)
payments_wallet = LocalWallet(
    PrivateKey(bytes.fromhex(os.getenv("NILLION_NILCHAIN_PRIVATE_KEY_0"))),
    prefix="nillion",
)

Configuration for the program to use. Change the contents of the below cell to use another program. 

In [10]:
program_name = "double"
program_mir_path = f"binaries/double.nada.bin"

As a baseline, let's ensure that the user can store the program and run it on their own secret. 

In [ ]:
import os

os.environ['RUST_BACKTRACE'] = '1'

print("The patient is storing the program on the network")

async def store_program():
    receipt_store_program = await get_quote_and_pay(
        patient_client,
        nillion.Operation.store_program(program_mir_path),
        payments_wallet,
        payments_client,
        cluster_id,
    )

    # Store the program
    action_id = await patient_client.store_program(
        cluster_id, program_name, program_mir_path, receipt_store_program
    )
    return action_id

action_id = await store_program()
print(f"Action ID: {action_id}")

print("The patient is storing a secret on the network")

program_id = f"{patient_user_id}/{program_name}"

new_secret = nillion.NadaValues(
    {
        "foo": nillion.SecretInteger(2),
    }
)

# Set permissions for the client to compute on the program
permissions = nillion.Permissions.default_for_user(patient_user_id)
permissions.add_compute_permissions({patient_user_id: {program_id}})

# Pay for and store the secret in the network and print the returned store_id
receipt_store = await get_quote_and_pay(
    patient_client,
    nillion.Operation.store_values(new_secret, ttl_days=5),
    payments_wallet,
    payments_client,
    cluster_id,
)
# Store a secret
store_id = await patient_client.store_values(
    cluster_id, new_secret, permissions, receipt_store
)

print("The patient is running the program on the secret..")

party_name = "Party1"
compute_bindings = nillion.ProgramBindings(program_id)
compute_bindings.add_input_party(party_name, patient_party_id)
compute_bindings.add_output_party(party_name, patient_party_id)
computation_time_secrets = nillion.NadaValues({})

# Pay for the compute
receipt_compute = await get_quote_and_pay(
    patient_client,
    nillion.Operation.compute(program_id, computation_time_secrets),
    payments_wallet,
    payments_client,
    cluster_id,
)

# Compute on the secret
compute_id = await patient_client.compute(
    cluster_id,
    compute_bindings,
    [store_id],
    computation_time_secrets,
    receipt_compute,
)

# 8. Return the computation result
print(f"The computation was sent to the network. compute_id: {compute_id}")
while True:
    compute_event = await patient_client.next_compute_event()
    if isinstance(compute_event, nillion.ComputeFinishedEvent):
        print(f"✅  Compute complete for compute_id {compute_event.uuid}")
        print(f"🖥️  The result is {compute_event.result.value}")
        break

Have Monadic store the program on the network

TODO: Also have Snipper save the program and replicate the computation

In [ ]:
async def store_program():
    receipt_store_program = await get_quote_and_pay(
        monadic_client,
        nillion.Operation.store_program(program_mir_path),
        payments_wallet,
        payments_client,
        cluster_id,
    )

    # Store the program
    action_id = await monadic_client.store_program(
        cluster_id, program_name, program_mir_path, receipt_store_program
    )
    return action_id

# Use this in your Jupyter notebook cell
action_id = await store_program()
print(f"Action ID: {action_id}")



Have the Patient store a secret on the network such that:
- The Patient has all the regular persmissions
- Monadic is able to modify permissions and compute on the secret
- Snipper is able to compute on the secret

TODO: Establish a chain by having Monadic actually assign compute access to Snipper

In [ ]:
program_id = f"{monadic_user_id}/{program_name}"

new_secret = nillion.NadaValues(
    {
        "foo": nillion.SecretInteger(2),
    }
)

# Set permissions for the client to compute on the program
permissions = nillion.Permissions.default_for_user(patient_user_id)
permissions.add_update_permissions(set([monadic_user_id]))
permissions.add_compute_permissions({monadic_user_id: {program_id}})
permissions.add_compute_permissions({snipper_user_id: {program_id}})

# Pay for and store the secret in the network and print the returned store_id
receipt_store = await get_quote_and_pay(
    patient_client,
    nillion.Operation.store_values(new_secret, ttl_days=5),
    payments_wallet,
    payments_client,
    cluster_id,
)
# Store a secret
store_id = await patient_client.store_values(
    cluster_id, new_secret, permissions, receipt_store
)

Have Monadic run the stored program on the stored secret

TODO: Have Snipper run the stored program on the stored secret

In [ ]:
import os

os.environ['RUST_BACKTRACE'] = 'full'

party_name = "Party1"
compute_bindings = nillion.ProgramBindings(program_id)
compute_bindings.add_input_party(party_name, monadic_party_id)
compute_bindings.add_output_party(party_name, monadic_party_id)

computation_time_secrets = nillion.NadaValues({})

# Pay for the compute
receipt_compute = await get_quote_and_pay(
    monadic_client,
    nillion.Operation.compute(program_id, computation_time_secrets),
    payments_wallet,
    payments_client,
    cluster_id,
)

# Compute on the secret
compute_id = await monadic_client.compute(
    cluster_id,
    compute_bindings,
    [store_id],
    computation_time_secrets,
    receipt_compute,
)

# 8. Return the computation result
print(f"The computation was sent to the network. compute_id: {compute_id}")
while True:
    compute_event = await monadic_client.next_compute_event()
    if isinstance(compute_event, nillion.ComputeFinishedEvent):
        print(f"✅  Compute complete for compute_id {compute_event.uuid}")
        print(f"🖥️  The result is {compute_event.result.value}")
        break

Now have Snipper run the program

In [ ]:
party_name = "Party1"
compute_bindings = nillion.ProgramBindings(program_id)
compute_bindings.add_input_party(party_name, snipper_party_id)
compute_bindings.add_output_party(party_name, snipper_party_id)

computation_time_secrets = nillion.NadaValues({})

# Pay for the compute
receipt_compute = await get_quote_and_pay(
    snipper_client,
    nillion.Operation.compute(program_id, computation_time_secrets),
    payments_wallet,
    payments_client,
    cluster_id,
)

# Compute on the secret
compute_id = await snipper_client.compute(
    cluster_id,
    compute_bindings,
    [store_id],
    computation_time_secrets,
    receipt_compute,
)

# 8. Return the computation result
print(f"The computation was sent to the network. compute_id: {compute_id}")
while True:
    compute_event = await snipper_client.next_compute_event()
    if isinstance(compute_event, nillion.ComputeFinishedEvent):
        print(f"✅  Compute complete for compute_id {compute_event.uuid}")
        print(f"🖥️  The result is {compute_event.result.value}")
        break